stac attempt following toward data science article 2/14

In [1]:
from pathlib import Path
import stackstac
import glob
import shutil
import pystac
from itertools import groupby

catalog = pystac.Catalog(
    id = 'RETREAT data, Langtang',
    description = 'RETREAT velocity data for glaciers in Langtang region of Nepal/Tibet',
    stac_extensions = ['https://stac-extensions.github.io/projection/v1.0.0/schema.json']
)

In [2]:
from shapely.geometry import Polygon, mapping
import rasterio as rio
import os

In [3]:
def get_bbox_and_footprint(dataset):
    #create boundingbox, will depent on if it comes from rasterio or rioxarray
    
    bounds = dataset.bounds
    
    if isinstance(bounds, rio.coords.BoundingBox):
        bbox = [bounds.left, bounds.bottom, bounds.right, bounds.top]
    else:
        bbox = [float(f) for f in bounds()]
    
    #create footprint
    footprint = Polygon([
        [bbox[0], bbox[1]],
        [bbox[0], bbox[3]],
        [bbox[2], bbox[3]],
        [bbox[2], bbox[1]]
    ])
    return bbox, mapping(footprint)

    return bbox, mapping(footprint)

In [4]:
import pandas as pd
from rasterio.warp import transform_bounds, transform_geom

#get list of files
folder = Path('/uufs/chpc.utah.edu/common/home/u1269862/2023/retreat/data/')

In [5]:
folder

PosixPath('/uufs/chpc.utah.edu/common/home/u1269862/2023/retreat/data')

In [6]:
files_ls = os.listdir(folder)

In [7]:
files_ls = [x for x in files_ls if not '.ipynb_checkpoints' in x]

In [8]:
files_ls[0][:10]

'10_019_001'

Need to group the files by footprint/ tile

In [9]:
#make a list of all the diff footprint ids
footprint_ids = set([x[:10] for x in files_ls])

In [10]:
footprint_ids 

{'10_019_001',
 '10_019_010',
 '10_019_019',
 '10_019_030',
 '10_019_043',
 '10_019_044',
 '10_019_045',
 '10_019_066',
 '10_019_069',
 '10_019_085',
 '10_085_002',
 '10_085_005',
 '10_085_015',
 '10_085_017',
 '10_085_028',
 '10_085_038',
 '10_085_053'}

make a dir for each tile:

In [11]:
!pwd

/uufs/chpc.utah.edu/common/home/u1269862/2023/retreat/code


In [12]:
gen_path = '/uufs/chpc.utah.edu/common/home/u1269862/2023/retreat/data/'
for element in footprint_ids:
    full_path = os.path.join(gen_path, element)
    os.makedirs(full_path, exist_ok=True)
    

Now need to move files into appropriate sub-dirs:

In [13]:
def move_files_to_tile_subdir(tile):
    
    for data in glob.glob(f'{gen_path}{tile}*'):
        shutil.move(data, f'{gen_path}/{tile}/')
#https://stackoverflow.com/questions/28913088/moving-files-with-wildcards-in-python   

In [14]:
for tile in footprint_ids:
    move_files_to_tile_subdir(tile)

In [15]:
move_files_to_tile_subdir('10_019_001')

In [16]:
#this will group all the files in the dir by the footprint ID (first 10 elements)
grouped = [list(g) for k,g in groupby(files_ls, lambda  s: s[:10])]

In [19]:
dis_angs[0].stem[:10]

'10_019_001'

In [20]:
dis_angs = list(folder.rglob('*-dis_angle*.tif'))

In [21]:
for dis_ang in dis_angs:
    
    #open file with rasterio
    ds = rio.open(dis_ang)
    
    bbox, footprint = get_bbox_and_footprint(ds)
    
    #project to wgs84, optain in geometric coords
    geo_bounds = transform_bounds(ds.crs, 'EPSG:4326', *bbox)
    geo_footprint = transform_geom(ds.crs, 'EPSG:4326', footprint)
    
    #properties
    idx = dis_ang.stem[:29]
    date = pd.to_datetime(dis_ang.stem[11:19])
    tile = dis_ang.stem[:10]
    
    item = pystac.Item(
        id = idx,
        geometry = geo_footprint,
        bbox = geo_bounds,
        datetime = date,
        stac_extensions = ['https://stac-extensions.github.io/projection/v1.0.0/schema.json'],
        properties = dict(
            tile=tile
        )
    )
    catalog.add_item(item)

print(len(list(catalog.get_items())))


87


Now, add assets

In [22]:
from pystac.extensions.projection import AssetProjectionExtension

for dis_ang in dis_angs:
    
    idx = dis_ang.stem[:29]
    item = catalog.get_item(idx)
    
    #as before, open both with rasterio to get bbox, footprint
    ds = rio.open(dis_ang)
    bbox, footprint = get_bbox_and_footprint(ds)
    
    item.add_asset(
        key = 'dis_angle',
        asset = pystac.Asset(
            href = dis_ang.as_posix(),
            media_type = pystac.MediaType.GEOTIFF
        )
    )
    #extend the asset wtih extension projection
    asset_ext = AssetProjectionExtension.ext(item.assets['dis_angle'])
    asset_ext.epsg = ds.crs.to_epsg()
    asset_ext.shape = ds.shape
    asset_ext.bbox = bbox
    asset_ext.geometry = footprint
    asset_ext.transform = [float(getattr(ds.transform, letter)) for letter in 'abcdef']

Now, save:

In [23]:
catalog.normalize_hrefs('/uufs/chpc.utah.edu/common/home/u1269862/retreat/stac_catalog')
catalog.save()

## Testing

In [24]:
import json

cat = pystac.Catalog.from_file('/uufs/chpc.utah.edu/common/home/u1269862/retreat/stac_catalog/catalog.json')
item = list(cat.get_items())[-1]

In [25]:
item

ID: 10_085_038_20181101-dis_angle
"Bounding Box: [83.8342115692863, 27.312423021686225, 86.8650951540967, 29.401377042492932]"
Datetime: 2018-11-01 00:00:00+00:00
tile: 10_085_038
datetime: 2018-11-01T00:00:00Z
stac_extensions: ['https://stac-extensions.github.io/projection/v1.0.0/schema.json']
https://stac-extensions.github.io/projection/v1.0.0/schema.json
href: /uufs/chpc.utah.edu/common/home/u1269862/2023/retreat/data/10_085_038/10_085_038_20181101-dis_angle+S1_20181026T122209_C471_1_9_1_9_1_9_S1_20181107T122209_CF41_1_9_1_9_1_9+250-50_50-10_0.00-0.08_2_geo_filtered_corrected.tif
Media type: image/tiff; application=geotiff
Owner:
proj:epsg: 32645


See if it works using stackstac (copied this fn directly from tds article)

In [30]:
def search_items(catalog, tiles, start_date, end_date, utc=True):

    # convert tiles to a list (if it is not)
    tiles = [tiles] if not isinstance(tiles, list) else tiles

    start_date = pd.to_datetime(start_date, utc=utc)
    end_date = pd.to_datetime(end_date, utc=utc)

    items = [item for item in catalog.get_items() if item.datetime > start_date and item.datetime < end_date and item.properties['tile'] in tiles]

    items = sorted(items, key=lambda x: x.datetime)

    return items

items = search_items(cat, [list(footprint_ids)[0]], '2014-01-01','2022-01-01')
len(items)

5

In [31]:
cube = stackstac.stack(
    items = [item.to_dict() for item in items])
   

In [32]:
cube

<xarray.DataArray 'stackstac-dddd715d2e2d5502956abbd32e60a6dc' (time: 5,
                                                                band: 1,
                                                                y: 1180, x: 1478)>
dask.array<fetch_raster_window, shape=(5, 1, 1180, 1478), dtype=float64, chunksize=(1, 1, 1024, 1024), chunktype=numpy.ndarray>
Coordinates:
  * time       (time) datetime64[ns] 2015-03-18 2015-05-05 ... 2015-07-10
    id         (time) <U29 '10_019_019_20150318-dis_angle' ... '10_019_019_20...
  * band       (band) <U9 'dis_angle'
  * x          (x) float64 1.348e+05 1.35e+05 1.352e+05 ... 4.3e+05 4.302e+05
  * y          (y) float64 3.313e+06 3.313e+06 3.313e+06 ... 3.078e+06 3.077e+06
    tile       <U10 '10_019_019'
    proj:epsg  int64 32645
    epsg       int64 32645
Attributes:
    spec:        RasterSpec(epsg=32645, bounds=(134800.0, 3077200.0, 430400.0...
    crs:         epsg:32645
    transform:   | 200.00, 0.00, 134800.00|\n| 0.00,-200.00, 3313200.00|\n| 0...
    resolution:  200.0